In [1]:
import random, numpy

In [2]:
def get_general_dirichlet_domain(generator, base):
    
    n = len(base[0])
    G = PermutationGroup(generator)
    inequalities = []

    for i in range(n):
        
        vector = base[i]
        partition = {}
        for j in range(n):
            if vector[j] in partition.keys():
                partition[vector[j]].append(j+1)
            else:
                partition[vector[j]] = [j+1]
        
        H = G
        for subset in partition:
            H = H.stabilizer(partition[subset], "OnSets")
        
        if H.cardinality() == G.cardinality():
            continue
        
        R = [x.Representative() for x in libgap.Group(G.gens()).RightCosets(libgap.Group(H.gens()))]
        
        for coset in R:
            lista = [0 for i in range(n+1)]
            for i in range(n):
                lista[i+1] = lista[i+1] + vector[i]
                lista[libgap.OnPoints(i+1, coset)] = lista[libgap.OnPoints(i+1, coset)] - vector[i]
            inequalities.append(tuple(lista))
            
            lista = [0 for i in range(n+1)]
            for i in range(n):
                lista[i+1] = lista[i+1] + vector[i]
                lista[libgap.OnPoints(i+1, libgap.Inverse(coset))] = lista[libgap.OnPoints(i+1, libgap.Inverse(coset))] - vector[i]
            inequalities.append(tuple(lista))
        G = H
    
    for i in range(n): #seteamos todas las variables >=0
        vector = [0 for i in range(n+1)]
        vector[i+1] = 1
        inequalities.append(tuple(vector))
    
    F = Polyhedron(ieqs = inequalities)
    return F


In [3]:
def get_rays_orbits(n, generador, rayos):
    
    I = IntegerVectorsModPermutationGroup(PermutationGroup(generador))
    orbitas = {}
    for i in range(len(rayos)):
        
        representante = I.retract(rayos[i])
        if (representante in orbitas.keys()) == True:
            orbitas[representante] = orbitas[representante] + [rayos[i]]
            continue
        
        orbitas[representante] = [rayos[i]]
            
    return orbitas

In [4]:
def generate_bases(dim, n):
    #generate n bases of R^dim
    random.seed(33)
    base = [[[0 for i in range(dim)] for j in range(dim)] for k in range(n)]
    k = 0
    while k < n:
        for i in range(dim):
            for j in range(dim):
                base[k][i][j] = random.choice([0, 1])
        if abs(numpy.linalg.det(base[k])) >= 0.9:
            k = k + 1
    return base

def print_matrix(matrix):
    print(numpy.array(matrix))
    return

In [5]:
#El output es una lista de tuplas que contienen un grupo y la dimensión del espacio en el que actúan
def generar_grupos(card_max, dim_max):
    lista_grupos=[]
    
    #Grupos S_n
    continuar=True
    i=2
    while continuar:
        grupo=SymmetricGroup(i)
        if grupo.cardinality()<=card_max and i<=dim_max:
            lista_grupos.append((grupo,i))
        else:
            continuar=False
        i+=1
        
    #Grupos alternantes
    continuar=True
    i=2
    while continuar:
        grupo=AlternatingGroup(i)
        if grupo.cardinality()<=card_max and i<=dim_max:
            lista_grupos.append((grupo,i))
        else:
            continuar=False
        i+=1
    
    #Grupos dihedrales
    continuar=True
    i=2
    while continuar:
        grupo=DihedralGroup(i)
        if grupo.cardinality()<=card_max and i<=dim_max:
            lista_grupos.append((grupo,i))
        else:
            continuar=False
        i+=1
    
    #Grupos cíclicos
    continuar=True
    i=2
    while continuar:
        grupo=CyclicPermutationGroup(i)
        if grupo.cardinality()<=card_max and i<=dim_max:
            lista_grupos.append((grupo,i))
        else:
            continuar=False
        i+=1
    
    #Grupos Dicíclicos
    continuar=True
    i=2
    while continuar:
        grupo=DiCyclicGroup(i)
        if grupo.cardinality()<=card_max and grupo.degree()<=dim_max:
            lista_grupos.append((grupo,grupo.degree()))
        else:
            continuar=False
        i+=1
        
    #Grupos semi dihedrales
    continuar=True
    i=4
    while continuar:
        grupo=SemidihedralGroup(i)
        if grupo.cardinality()<=card_max and grupo.degree()<=dim_max:
            lista_grupos.append((grupo,grupo.degree()))
        else:
            continuar=False
        i+=1
        
    #Split Metacyclic Group
    continuar = True
    i=2
    j=4
    while continuar:
        if i.is_prime():
            grupo=SplitMetacyclicGroup(i,j)
            if grupo.cardinality()<=card_max and grupo.degree()<=dim_max:
                lista_grupos.append((grupo,grupo.degree()))
                j+=1
            elif j==4:
                continuar=False
            else:
                i+=1
                j=4
        else:
            i+=1
    
    #Mathieu Groups
    for i in [9,10,11,12,21,22,23]:
        grupo=MathieuGroup(i)
        if grupo.cardinality()<=card_max and i<=dim_max:
            lista_grupos.append((grupo,grupo.degree()))
    
    return lista_grupos

In [6]:
def experiment(generator, dim, base):
    
    orbitas = []
    F = get_general_dirichlet_domain(generator, base)
    rayos = F.rays()
    orbitas = get_rays_orbits(dim, generator, rayos)
    largos = [len(orbitas[orbita]) for orbita in orbitas.keys()]
    largos.sort(reverse = True)
    suma = 0
    for u in largos:
        suma = suma + u
    return list([largos[0], largos[len(largos) - 1], suma, len(largos), len(F.Hrepresentation())])

In [28]:
grupos = generar_grupos(100000, 20)
num_bases = 10
bases = [generate_bases(i+1, num_bases) for i in range(20)] #bases[i][j] es la j-ésima base de dimensión i+1

#array = ['Generador', 'dimension', 'base', 'orbita mayor', 'orbita menor', 'suma orbitas', 'numero orbitas', 'numero desigualdades']
array = []
#for i in range(len(grupos)):
for i in range(10):
    if i == 7 or i == 14:
        continue
    generador = grupos[i][0].gens()
    dimension = grupos[i][1]
    for j in range(num_bases):
        stats = [str(generador), dimension, j]
        stats = stats + experiment(generador, dimension, bases[dimension - 1][j])
        array.append(stats)
        
array = numpy.array(array)

print(array)

[['[(1,2)]' '2' '0' '1' '1' '2' '2' '2']
 ['[(1,2)]' '2' '1' '1' '1' '2' '2' '2']
 ['[(1,2)]' '2' '2' '1' '1' '2' '2' '2']
 ['[(1,2)]' '2' '3' '1' '1' '2' '2' '2']
 ['[(1,2)]' '2' '4' '1' '1' '2' '2' '2']
 ['[(1,2)]' '2' '5' '1' '1' '2' '2' '2']
 ['[(1,2)]' '2' '6' '1' '1' '2' '2' '2']
 ['[(1,2)]' '2' '7' '1' '1' '2' '2' '2']
 ['[(1,2)]' '2' '8' '1' '1' '2' '2' '2']
 ['[(1,2)]' '2' '9' '1' '1' '2' '2' '2']
 ['[(1,2,3), (1,2)]' '3' '0' '1' '1' '3' '3' '3']
 ['[(1,2,3), (1,2)]' '3' '1' '1' '1' '3' '3' '3']
 ['[(1,2,3), (1,2)]' '3' '2' '1' '1' '3' '3' '3']
 ['[(1,2,3), (1,2)]' '3' '3' '1' '1' '3' '3' '3']
 ['[(1,2,3), (1,2)]' '3' '4' '1' '1' '3' '3' '3']
 ['[(1,2,3), (1,2)]' '3' '5' '1' '1' '3' '3' '3']
 ['[(1,2,3), (1,2)]' '3' '6' '1' '1' '3' '3' '3']
 ['[(1,2,3), (1,2)]' '3' '7' '1' '1' '3' '3' '3']
 ['[(1,2,3), (1,2)]' '3' '8' '1' '1' '3' '3' '3']
 ['[(1,2,3), (1,2)]' '3' '9' '1' '1' '3' '3' '3']
 ['[(1,2,3,4), (1,2)]' '4' '0' '1' '1' '4' '4' '4']
 ['[(1,2,3,4), (1,2)]' '4' '1' '1' '1'